# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

# from google.colab import drive
# drive.mount('/content/drive/')
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
iris = datasets.load_iris()
X_train, X_test, Y_train, Y_test = train_test_split(
    iris.data, iris.target, test_size=0.33, random_state=239)


class Node:
    def __init__(self, chances=None):
        self.chances = chances
        self.t = -1
        self.depth = 1
        self.bad = False

    def _make_adjacent_node(self, chances):
        self.depth += 1
        return Node(chances)


class Tree:
    def __init__(self, max_depth=5, max_leafs=5, min_leaf_obj=5):
        self.max_depth = max_depth
        self.min_leaf_obj = min_leaf_obj
        self.max_leafs = max_leafs

    def _chance(self, Y): # Считаем долю каждого типа Y от общего числа
        return np.array([len(Y[Y == Y_type]) / len(Y) for Y_type in set(self.Y)])

    @staticmethod
    def _information_gain(Rv, Rleft, Rright): # Просто вычисляем по формуле
        return entropy(Rv) - ((len(Rleft) / len(Rv) * entropy(Rleft)) + (len(Rright) / len(Rv) * entropy(Rright)))

    def _split(self, X, Y, node):
        # Проверяем условия на кол-во листов, глубину, корректность Y
        if node.depth >= self.max_depth or len(X) < self.max_leafs or len(set(Y)) == 1:
            node.bad = True
            return

        best_split_j, best_t, best_ig = -1, -1, -1
        best_left_part, best_right_part = map(np.copy, (np.array([]), np.array([])))
        x1, x2 = map(np.copy, (np.array([]), np.array([])))

        for j, x_col in enumerate(X.T): # Проходимся по столбцам
            for t in x_col: # проходимся по элементам столбцов
                left_part, right_part = Y[x_col < t], Y[x_col >= t] # делим Y на две части по предикату t
                if len(left_part) == 0 or len(right_part) == 0: # если части пустые, то пропускаем их
                    continue

                ig = self._information_gain(Y, left_part, right_part) # Считаем прирост информации
                if ig > best_ig: # Если новое разбиение более информативно, то пересчитываем все переменные
                    best_t, best_ig = t, ig # предикат, прирост информации
                    best_left_part, best_right_part = left_part, right_part # части
                    best_split_j = j # индекс столбца
                    x1 = X[x_col >= best_t] # в правую часть строки из X, в которых столбец был >= предиката
                    x2 = X[x_col < best_t] # в правую часть строки из X, в которых столбец был < предиката
            # Проверяем условие на минимальное число объектов в листе
            if len(x1) < self.min_leaf_obj or len(x2) < self.min_leaf_obj:
                node.bad = True
                return

            # Добавляем вершины слева и справа от текущей
            node.left = node._make_adjacent_node(self._chance(best_left_part))
            node.right = node._make_adjacent_node(self._chance(best_right_part))
            node.j, node.t =  best_split_j, best_t
            self._split(x1, best_right_part, node.right) # Запустили для правой части
            self._split(x2, best_left_part, node.left) # Запустили для левой

    def _predict_node(self, X, node):
        if node.bad:
            return node.chances
        elif X[node.j] > node.t:
            return self._predict_node(X, node.right)
        else:
            return self._predict_node(X, node.left)

    def fit(self, X, Y):
        self.Y = Y
        self.tree = Node(self._chance(Y))
        self._split(np.asarray(X), Y, self.tree)

    def predict(self, X):
        return [np.argmax(self._predict_node(x, self.tree)) for x in X]


tree = Tree()
tree.fit(X_train, Y_train)

Y_pred = tree.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print(f'Точность метода: {accuracy * 100}%')

Точность метода: 98.0%


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
class RandomForest:
  def __init__(self, number_of_trees=5, max_depth=5, min_samples_leaf=5, random_state=239):
    self.number_of_trees = number_of_trees
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf
    self.random_state = random_state
    np.random.seed(random_state) # Позволяет делать одинаковый choice и permutation
    self.trees = []

  @staticmethod
  def _boostrap(X, Y): # Просто делаем случайный выбор заданного кол-ва объектов с возможностью перевыбора
    n_rows, n_cols = X.shape
    samples = np.random.choice(n_rows, size=n_rows, replace=True) 
    return X.iloc[samples], Y.iloc[samples]

  def fit(self, X, Y):
    if len(self.trees) > 0:
      self.trees = []
    for _ in range(self.number_of_trees):
      tree = DecisionTreeClassifier(criterion="entropy", # Критерий для измерения качества разбиения
                                    random_state=self.random_state,
                                    min_samples_split=self.min_samples_leaf,
                                    max_depth=self.max_depth,
                                    max_features="sqrt")
      sample_x, sample_y = self._boostrap(X, Y)
      tree.fit(sample_x, sample_y)
      self.trees += [tree]

  # def fit_custom(self, X, Y):
  #   if len(self.trees) > 0:
  #     self.trees = []
  #   for _ in range(self.number_of_trees):
  #     tree = Tree(min_leaf_obj=self.min_samples_leaf,
  #                 max_depth=self.max_depth)
  #     sample_x, sample_y = self._boostrap(X, Y)
  #     tree.fit(sample_x, sample_y)
  #     self.trees += [tree]

  def predict(self, X):
      Y = np.array([tree.predict(X)  for tree in self.trees])

      predictions = np.array([])
      for prediction in Y.T:
        counter = Counter(prediction)
        predictions = np.append(predictions, counter.most_common(1)[0][0])
      return predictions
  
  def attribute_value(self, old_accuracy, X_test, Y_test): # Считает "важность" признаков
    predictions = [] # list tuple-ов, на первом месте посчитанная важность признака, на втором - его название
    for column in X_test:
      X_copy = X_test.copy()
      X_copy[column] = np.random.permutation(X_copy[column].values) # делаем случайную перестановку
      predicted_score = accuracy_score(Y_test, self.predict(X_copy))
      predictions.append((abs(predicted_score - old_accuracy) / old_accuracy, column)) # |старая_точность - предсказанная_точность| / старая_точность
    predictions = sorted(predictions, key=lambda x: x[0], reverse=True)
    return predictions

In [ ]:
# Заменяет в DataFrame значения колонки column_name на среднеарифметическое соответсвующего значения из number_column
def column_mean_replacer(X, column_name, number_column):
  for x_u in X[column_name].unique(): # Проходимся только по уникальным значениям
    mean_val = X[X[column_name] == x_u][number_column].mean() # Считаем для уникального(неповторяющегося) значения x_u среднеарифметическое (mean_val) колонки number_column
    X[column_name].replace({x_u: mean_val}, inplace=True) # Заменяем уникальное значение x_u в колонке column_name на mean_val

In [ ]:
chur_table = pd.read_csv('/content/drive/MyDrive/churn.csv')
chur_table.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
chur_table = chur_table.drop(["Surname", "CustomerId"], axis=1)
column_mean_replacer(chur_table, 'Geography','Exited')
chur_table = chur_table.replace({'Gender': {'Female': 1, 'Male' : 0}})

X = chur_table.iloc[:, 1:-1]
Y = chur_table["Exited"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=239, test_size=0.2)

forest = RandomForest(number_of_trees=5, max_depth=6)
forest.fit(X_train, Y_train)
Y_pred = forest.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)

forest_attributes = forest.attribute_value(accuracy, X_test, Y_test)

pd.DataFrame(forest_attributes)

,0,1
0,0.082444,Age
1,0.034994,NumOfProducts
2,0.030842,IsActiveMember
3,0.019573,Geography
4,0.011862,Balance
5,0.002966,Gender
6,0.002372,HasCrCard
7,0.002372,EstimatedSalary
8,0.001186,Tenure
9,0.000593,CreditScore


Из полученного датафрейма можно сделать вывод, что наиболее значимым фактором является возраст (Age) клиента.

In [ ]:
best_accuracy, number_of_trees, max_depth, min_samples_leaf = -1, -1, -1, -1

for number_of_trees in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]:
  for max_depth in [3, 6, 9, 12, 15, 18, 21]:
    for min_samples_leaf in [1.0, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]:
      forest = RandomForest(number_of_trees, max_depth, min_samples_leaf)
      forest.fit(X_train, Y_train)
      Y_pred = forest.predict(X_test)
      accuracy = accuracy_score(Y_test, Y_pred)
      if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_number_of_trees = number_of_trees
        best_max_depth = max_depth
        best_min_samples_leaf = min_samples_leaf


print(f'Лучшая точность: {best_accuracy*100}%')
print(f'Лучшее колличествово деревьев: {number_of_trees}')
print(f'Лучшая максимальная глубина деревьев: {max_depth}')
print(f'Лучшая кол-во объектов в листе: {min_samples_leaf}')

Лучшая точность: 86.65%
Лучшее колличествово деревьев: 55
Лучшая максимальная глубина деревьев: 21
Лучшая кол-во объектов в листе: 21
